In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

df = pd.read_stata('Returns/MLB/MLBHomeReturnsnovig.dta')
df['AwayValue'] = 0
df['AwayRev'] = 0
df['HomeValue'] = 0
df['HomeRev'] = 0

dropteams = ['American', 'National', 'AL', 'NL', 'STF', 'ALT', 'STF']

df = df[~df['awayteam'].isin(dropteams)]
df = df[~df['hometeam'].isin(dropteams)]
teams = pd.concat([df['awayteam'],df['hometeam']]).unique()

teamDict = {}
for team in teams:
    teamDict[team] = {'pfor':[], 'pagainst':[], 'lagreturns':0, 'seasonret': 0}

df['year'] = df['date'] //10000

['ATL' 'NYM' 'CLE' 'COL' 'NYY' 'WSH' 'PIT' 'LAD' 'CHC' 'DET' 'SEA' 'OAK'
 'KC' 'TB' 'MIL' 'TEX' 'TOR' 'PHI' 'MIN' 'HOU' 'BOS' 'SF' 'CHW' 'SD' 'CIN'
 'BAL' 'SLC' 'LAA' 'MIA' 'ARI']


In [2]:
values = pd.read_excel('MLB Size.xlsx', sheet_name= 'Val', index_col=0)
revs = pd.read_excel('MLB Size.xlsx', sheet_name='Rev', index_col=0)

In [3]:
year = 2009
GAMES = 2430
N = 5

In [4]:
for index, row in df.iterrows():
    awayteam = row['awayteam']
    hometeam = row['hometeam']

    curyear = row['year']
    
    df.loc[[index], ['HomeValue']] = values.at[hometeam, year]
    df.loc[[index], ['HomeRev']] = revs.at[hometeam, year]

    df.loc[[index], ['AwayValue']] = values.at[awayteam, year]
    df.loc[[index], ['AwayRev']] = revs.at[awayteam, year]    

In [5]:
df['HomeValue'] = (df['HomeValue'] - df['HomeValue'].mean()) / df['HomeValue'].std()
df['HomeRev'] = (df['HomeRev'] - df['HomeRev'].mean()) / df['HomeRev'].std()
df['AwayValue'] = (df['AwayValue'] - df['AwayValue'].mean()) / df['AwayValue'].std()
df['AwayRev'] = (df['AwayRev'] - df['AwayRev'].mean()) / df['AwayRev'].std()

In [6]:
df['HomeChar'] = df['HomeValue'] + df['HomeRev']
df['AwayChar'] = df['AwayValue'] + df['AwayValue']

In [7]:
chardf = df.copy(deep=True)
chardf.drop(chardf[(chardf.year == 2009)].index, inplace= True)

In [8]:
chardf['Size'] = df['HomeChar'] - df['AwayChar']
chardf['games'] = chardf.groupby('date')['date'].transform('size')
chardf['ranks'] = chardf.groupby('date')['Size'].rank()
test = chardf.drop(chardf[chardf.games < N].index, inplace = True)

chardf['thresh'] = chardf.ranks / chardf.games
chardf['buy'] = chardf['thresh'].ge(.8)
chardf['sell'] = chardf['thresh'].le(.2)
#chardf.to_stata('MLBSpreadChar.dta')

In [9]:
chardf['overSize'] = chardf['HomeChar'] + df['AwayChar']
chardf['overRanks'] = chardf.groupby('date')['overSize'].rank()

chardf['overthresh'] = chardf.overRanks / chardf.games
chardf['overBuy'] = chardf['overthresh'].ge(.8)
chardf['overSell'] = chardf['overthresh'].le(.2)

In [10]:
chardf['weight'] = chardf['buy'] / chardf['games'] - chardf['sell'] / chardf['games']
chardf['smallocreturn'] = chardf['weight'] * chardf['mlhomeOCreturn']
chardf['smallclosereturn'] = chardf['weight'] * chardf['mlhomeclosereturn']

chardf['overWeight'] = chardf['overBuy'] / chardf['games'] - chardf['overSell'] / chardf['games']
chardf['smalloverocreturn'] = chardf['overWeight'] * chardf['overOCreturn']
chardf['smalloverclosereturn'] = chardf['overWeight'] * chardf['overclosereturn']

chardf.to_stata('MLBsizeChar.dta')

In [11]:
weights  = chardf[chardf.weight != 0]

MLBmlsizeret = pd.DataFrame(weights.groupby('date')['smallocreturn'].sum())
MLBmlsizeret.rename(columns = {'smallocreturn':'mlocreturn'}, inplace=True)
MLBmlsizeret['mlclosereturns'] = weights.groupby('date')['smallclosereturn'].sum()
MLBmlsizeret['mlopenreturn'] = MLBmlsizeret['mlocreturn'] + MLBmlsizeret['mlclosereturns']
MLBmlsizeret['mltrading'] = MLBmlsizeret['mlocreturn'] - MLBmlsizeret['mlclosereturns']
means = MLBmlsizeret.mean()*GAMES * 2/5
std = MLBmlsizeret.std() * np.sqrt(GAMES) * np.sqrt(2/5)
print(means)
print(means / std)

returns = ['mlocreturn', 'mlclosereturns', 'mlopenreturn', 'mltrading']
tstats = {} 
for x in returns:
    tstats[x] = sm.ols(formula="{}~1".format(x), data = MLBmlsizeret).fit().tvalues['Intercept']
print(tstats)

mlocreturn       -0.314604
mlclosereturns    2.374545
mlopenreturn      2.059941
mltrading        -2.689148
dtype: float64
mlocreturn       -0.602431
mlclosereturns    0.432984
mlopenreturn      0.376892
mltrading        -0.484362
dtype: float64
{'mlocreturn': -0.921242601542379, 'mlclosereturns': 0.6621232511152155, 'mlopenreturn': 0.576346026805342, 'mltrading': -0.7406896300219754}


In [12]:
overweights  = chardf[chardf.overWeight != 0]
MLBoversizeret = pd.DataFrame(overweights.groupby('date')['smalloverocreturn'].sum())
MLBoversizeret.rename(columns = {'smalloverocreturn':'overocreturn'}, inplace=True)
MLBoversizeret['overclosereturns'] = overweights.groupby('date')['smalloverclosereturn'].sum() 
MLBoversizeret['overopenreturn'] = MLBoversizeret['overocreturn'] + MLBoversizeret['overclosereturns']
MLBoversizeret['overtrading'] = MLBoversizeret['overocreturn'] - MLBoversizeret['overclosereturns']

means = MLBoversizeret.mean()*GAMES * 2/5
std = MLBoversizeret.std() * np.sqrt(GAMES) * np.sqrt(2/5)
print(means)
print(means / std)


returns = ['overocreturn', 'overclosereturns', 'overopenreturn', 'overtrading']
tstats = {}
for x in returns:
    tstats[x] = sm.ols(formula="{}~1".format(x), data = MLBoversizeret).fit().tvalues['Intercept']
print(tstats)

#MLBoversizeret.to_stata('MLBSizeTrading.dta')

overocreturn        1.159200
overclosereturns   -3.905277
overopenreturn     -2.746078
overtrading         5.064477
dtype: float64
overocreturn        0.731282
overclosereturns   -0.686689
overopenreturn     -0.482334
overtrading         0.829260
dtype: float64
{'overocreturn': 1.1182820426355091, 'overclosereturns': -1.0500904447230333, 'overopenreturn': -0.737589444385228, 'overtrading': 1.268111133757384}
